# Import dependancies

In [59]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import select

# Create schema

In [60]:
class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"


# Create the connection engine

In [61]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///example.db", echo=False) #Get the database from this relative path, create if it doesnt exist

# Apply the schema through the connection engine

In [62]:
#Create all tables in the Base schema (if they dont exist)
Base.metadata.create_all(engine)

# Create some data to add

In [65]:
from sqlalchemy.orm import Session

#Use session as context manager to make it automatically close when finished.
with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")

    #Insert all the elements (if they exist they will still be inserted)
    #Though that may change if i added a primary key. TODO: Check
    session.add_all([spongebob, sandy, patrick])
    session.commit()

# Update an entity using the ORM

In [69]:
with Session(engine) as session:
    spongebob = User(
        id=1,
        name="new spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    session.merge(spongebob) #Merge updates the row if it already exists or inserts a new row if it doesn’t
    session.commit()
#Changes to elements are automatically tracked, so I Could've also just modified the original spongebob user and executed commit within a session context.

# Query using the ORM

In [70]:
with Session(engine) as session:
    users = select(User)

    for user in session.scalars(users): #could use session.scalars(users).one() to grab only one
        print(user)

User(id=1, name='new spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')
User(id=3, name='patrick', fullname='Patrick Star')
User(id=4, name='spongebob', fullname='Spongebob Squarepants')
User(id=5, name='sandy', fullname='Sandy Cheeks')
User(id=6, name='patrick', fullname='Patrick Star')
User(id=7, name='spongebob', fullname='Spongebob Squarepants')
User(id=8, name='sandy', fullname='Sandy Cheeks')
User(id=9, name='patrick', fullname='Patrick Star')
User(id=10, name='spongebob', fullname='Spongebob Squarepants')
User(id=11, name='sandy', fullname='Sandy Cheeks')
User(id=12, name='patrick', fullname='Patrick Star')


# Execute a query using normal SQL

In [16]:
from sqlalchemy import text
with engine.connect() as connection:
    result = connection.execute(text("SELECT name FROM sqlite_master WHERE type='table'")) #If you needed parameters aka a prepared statement, just use the second parameter of execute, values.
    for row in result:
        print(row)

2024-06-10 12:29:33,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-10 12:29:33,074 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table'
2024-06-10 12:29:33,075 INFO sqlalchemy.engine.Engine [cached since 1075s ago] ()
('user_account',)
('address',)
2024-06-10 12:29:33,078 INFO sqlalchemy.engine.Engine ROLLBACK
